# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('con').getOrCreate()

In [3]:
df = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [4]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [9]:
output = assembler.transform(df)

In [12]:
final_data = output.select('features','churn')

In [13]:
final_data.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [14]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
log_model = LogisticRegression(labelCol='churn')

In [17]:
fitted_model = log_model.fit(train_data)

In [19]:
training_summary = fitted_model.summary

In [24]:
training_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                635|                635|
|   mean|0.16692913385826771|0.13228346456692913|
| stddev|0.37320656700782473|  0.339065772018556|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [26]:
predictions_and_labels = fitted_model.evaluate(test_data)

In [27]:
predictions_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.13675887222991...|[0.53413653003898...|       0.0|
|[28.0,11245.38,0....|    0|[3.86668345117211...|[0.97950132722804...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.55846320606222...|[0.92814002696330...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.63175438598708...|[0.99035624709879...|       0.0|
|[29.0,11274.46,1....|    0|[4.53414824672617...|[0.98937799060351...|       0.0|
|[29.0,13240.01,1....|    0|[7.06864005160964...|[0.99914933407969...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.57949773379181...|[0.97286702773903...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.18090013829107...|[0.99793570510643...|       0.0|
|[30.0,10744.14,1....|    1|[1.53661362904036...|[0.82297190719114...|       0.0|
|[31.0,5387.75,0

In [29]:
my_eval=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [31]:
my_eval.evaluate(predictions_and_labels.predictions)

0.7433155080213903

In [32]:
final_lr_model = log_model.fit(final_data)

In [33]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [39]:
test = assembler.transform(new_customers)

In [43]:
results =final_lr_model.transform(test)

In [46]:
results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

